<a href="https://colab.research.google.com/github/martinpius/PYTORCH/blob/main/Residual_Network_Implementation_from_scratch_with_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount("/content/drive", force_remount = True)
try:
  COLAB = True
  import torch
  print(f"You are on Google CoLaB with Pytorch Version: {torch.__version__}")
except Exception as e:
  print(f"{type(e)}: {e}\n>>>please correct {type(e)} and reload")
  COLAB = False
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
def time_fmt(t: float = 123.891)->float:
  h = int(t / (60 * 60))
  m = int(t % (60 * 60) / 60)
  s = int(t % 60)
  return f"{h}: {m:>02}: {s:>05.2f}"
print(f">>>>time formating:\tpleas wait....\n>>>>time elapsed:\t{time_fmt()}")

Mounted at /content/drive
You are on Google CoLaB with Pytorch Version: 1.8.1+cu101
>>>>time formating:	pleas wait....
>>>>time elapsed:	0: 02: 03.00


In [5]:
#In this notebook we are going to impliment the residual network(resnet50, resnet101, resnet152) from scratch.
#Due to computational intensive we will freeze some layers and train on cifar10 data for the demo(few iterations)


In [73]:
import torch.nn as nn
import torch.optim as optim
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision.transforms import transforms
from tqdm import tqdm
import time, sys


In [135]:
#We will start by bulding convolutional block that will be reused over and over in the construction of residual blocks
class CNNBLOCK(nn.Module):
  def __init__(
        self, in_channels, out_channels, id_downsample=None, stride=1):
        super(CNNBLOCK, self).__init__()
        self.expansion = 4
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False)
        self.bnorm1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(
            out_channels,
            out_channels,
            kernel_size=3,
            stride=stride,
            padding=1,
            bias=False)
        self.bnorm2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(
            out_channels,
            out_channels * self.expansion,
            kernel_size=1,
            stride=1,
            padding=0,
            bias=False)
        self.bnorm3 = nn.BatchNorm2d(out_channels * self.expansion)
        self.relu = nn.ReLU()
        self.id_downsample = id_downsample
        self.stride = stride
  
  def forward(self, x):
    identity = x.clone()
    x = self.conv1(x)
    x = self.bnorm1(x)
    x = self.relu(x)
    x = self.conv2(x)
    x = self.bnorm2(x)
    x = self.relu(x)
    x = self.conv3(x)
    x = self.bnorm3(x)
    if self.id_downsample is not None:
      identity = self.id_downsample(identity)
    x += identity
    x = self.relu(x)
    return x


In [136]:
#We now construct the residual block using the above set of convolutions:

In [137]:
class ResBlock(nn.Module):
  def __init__(self, cnn_block, layers, image_channels, num_classes):
    super(ResBlock, self).__init__()
    self.in_channels = 64
    self.conv1 = nn.Conv2d(in_channels = image_channels, out_channels = 64, kernel_size = 7, stride = 2, padding = 3)
    self.bnorm = nn.BatchNorm2d(num_features = 64)
    self.relu = nn.ReLU()
    self.maxpool = nn.MaxPool2d(kernel_size = 3, stride = 2, padding = 1)
    self.res_layer1 = self.make_layers(cnn_block, layers[0], out_channels = 64, stride = 1)
    self.res_layer2 = self.make_layers(cnn_block, layers[1], out_channels = 128, stride = 2)
    self.res_layer3 = self.make_layers(cnn_block, layers[2], out_channels = 256, stride = 2)
    self.res_layer4 = self.make_layers(cnn_block, layers[3], out_channels = 512, stride = 2)
    self.avgpool = nn.AdaptiveAvgPool2d(output_size = (1,1))
    self.fc = nn.Linear(in_features = 512 * 4, out_features = num_classes)
  
  def forward(self, input_tensor):
    x = self.conv1(input_tensor)
    x = self.bnorm(x)
    x = self.relu(x)
    x = self.maxpool(x)
    x = self.res_layer1(x)
    x = self.res_layer2(x)
    x = self.res_layer3(x)
    x = self.res_layer4(x)
    x = self.avgpool(x)
    x = x.reshape(x.shape[0], -1)
    x = self.fc(x)
    return x

  def make_layers(self, block, num_residual_blocks, out_channels, stride):
    id_downsample = None
    layers = []
    if stride != 1 or self.in_channels != out_channels * 4:
      id_downsample = nn.Sequential(
                nn.Conv2d(
                    self.in_channels,
                    out_channels * 4,
                    kernel_size=1,
                    stride=stride,
                    bias=False
                ),
                nn.BatchNorm2d(out_channels * 4),
            )

    layers.append(block(self.in_channels, out_channels, id_downsample, stride))
    self.in_channels = out_channels * 4
    for i in range(num_residual_blocks - 1):
      layers.append(block(self.in_channels, out_channels))
    return nn.Sequential(*layers)

In [138]:
#We define our final class to create resnet50, resnet101 and resnet152 as follows:
class ResidualNet():

  def resnet50(self, channels = 3, num_classes = 1000):
    return ResBlock(CNNBLOCK, [3, 4, 6, 3],channels, num_classes)
  
  def resnet101(self, channels = 3, num_classes = 1000):
    return ResBlock(CNNBLOCK, [3, 4, 23, 3], channels, num_classes)
  
  def resnet152(self, channels = 3, num_classes = 1000):
    return ResBlock(CNNBLOCK, [3, 8, 36, 3], channels, num_classes)

  

In [139]:
#Testing our models on a simulated data to see if its gives the intended outputs:

In [146]:
my_resnet = ResidualNet()
def test():
    net1 = my_resnet.resnet50(3, 1000)
    net2 = my_resnet.resnet101(3,1000)
    net3 = my_resnet.resnet152(3,100)
    y1 = net1(torch.rand(2,3,224,224))
    y2 = net2(torch.rand(2,3,224,224))
    y3 = net3(torch.rand(2,3,224,224))
    print(f"output_shape for resnet50: {y1.size()}\noutput_shape for resnet101: {y2.size()}\noutput_shape for resnet152: {y3.size()}")


test()

output_shape for resnet50: torch.Size([2, 1000])
output_shape for resnet101: torch.Size([2, 1000])
output_shape for resnet152: torch.Size([2, 100])


In [147]:
#Model summary for resnet50, resnet101, resnet152


In [148]:
#Summary for resnet50
print(my_resnet.resnet50())

ResBlock(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (res_layer1): Sequential(
    (0): CNNBLOCK(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bnorm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bnorm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bnorm3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (id_downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
  

In [149]:
#Summary for resnet101
print(my_resnet.resnet101())

ResBlock(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (res_layer1): Sequential(
    (0): CNNBLOCK(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bnorm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bnorm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bnorm3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (id_downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
  

In [150]:
#Summary for resnet152:
print(my_resnet.resnet152())

ResBlock(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bnorm): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU()
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (res_layer1): Sequential(
    (0): CNNBLOCK(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bnorm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bnorm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bnorm3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU()
      (id_downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
  